In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.callbacks import EarlyStopping


In [46]:
def adjust_brightness_contrast(image, brightness=40, contrast=1.0):
    # Convert to float to prevent clipping
    img = image.astype(np.float32)
    # Adjust brightness and contrast
    img = img * contrast + brightness
    # Clip to keep pixel values between 0 and 255 and convert back to uint8
    img = np.clip(img, 0, 255).astype(np.uint8)
    return img

In [47]:
# Initialize MediaPipe Hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Path where images are stored for each letter in the ASL alphabet
data_dir = "~/asl_alphabet_train/asl_alphabet_train" # Change according to local dataset

landmark_data = []
labels = []

In [48]:
# Collect landmarks for each letter
for letter in os.listdir(data_dir):
    #if letter=="C":
    #    break
    letter_dir = os.path.join(data_dir, letter)
    for i, img_path in enumerate(os.listdir(letter_dir)):
        #if i >= 300:
        #    break
        img = cv2.imread(os.path.join(letter_dir, img_path))
        img = adjust_brightness_contrast(img, 40, 1)
        #img = adjust_brightness_contrast(img, 20, 0.7)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        
        # Check for hand landmarks and store them
        if results.multi_hand_landmarks:
            landmarks = []
            for lm in results.multi_hand_landmarks[0].landmark:
                landmarks.extend([lm.x, lm.y, lm.z])  # Flattened landmark vector
            landmark_data.append(landmarks)
            labels.append(letter)  # Store the label (e.g., "A", "B", etc.)
    print(letter)

A
B
C
D
del
E
F
G
H
I
J
K
L
M
N
nothing
O
P
Q
R
S
space
T
U
V
W
X
Y
Z


In [61]:
# Convert to arrays for model input
landmark_data = np.array(landmark_data)
labels = np.array(labels)

print(labels)
# Save the arrays to .npy files
np.save("landmark_data.npy", landmark_data)
np.save("labels.npy", labels)

# Normalize landmarks between 0 and 1
landmark_data = landmark_data / np.max(landmark_data)

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

['A' 'A' 'A' ... 'Z' 'Z' 'Z']


In [50]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(landmark_data, labels_categorical, test_size=0.2, random_state=42)

In [51]:
# Build 1D CNN model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [52]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(patience = 5, restore_best_weights=True)

# Train the model
model.fit(X_train[..., np.newaxis], y_train, epochs=40, batch_size=32, validation_data=(X_test[..., np.newaxis], y_test), callbacks=es)
#model.save("asl_sign_language_model.h5")
save_model(model, 'asl_sign_language_model.keras')

Epoch 1/40
1633/1633 [==============================] - 8s 5ms/step - loss: 0.9426 - accuracy: 0.7157 - val_loss: 0.1541 - val_accuracy: 0.9646
Epoch 2/40
1633/1633 [==============================] - 7s 5ms/step - loss: 0.2773 - accuracy: 0.9155 - val_loss: 0.1043 - val_accuracy: 0.9701
Epoch 3/40
1633/1633 [==============================] - 8s 5ms/step - loss: 0.2128 - accuracy: 0.9353 - val_loss: 0.0754 - val_accuracy: 0.9842
Epoch 4/40
1633/1633 [==============================] - 8s 5ms/step - loss: 0.1761 - accuracy: 0.9472 - val_loss: 0.0698 - val_accuracy: 0.9811
Epoch 5/40
1633/1633 [==============================] - 8s 5ms/step - loss: 0.1552 - accuracy: 0.9528 - val_loss: 0.0661 - val_accuracy: 0.9830
Epoch 6/40
1633/1633 [==============================] - 8s 5ms/step - loss: 0.1384 - accuracy: 0.9588 - val_loss: 0.0553 - val_accuracy: 0.9865
Epoch 7/40
1633/1633 [==============================] - 8s 5ms/step - loss: 0.1250 - accuracy: 0.9622 - val_loss: 0.0515 - val_accuracy:

In [53]:
# Load the trained model
#model = tf.keras.models.load_model("asl_sign_language_model.h5")
model = tf.keras.models.load_model("asl_sign_language_model.keras")

In [54]:
# Encode labels as integers and convert to categorical
labels = np.load("labels.npy")

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [55]:
# Initialize MediaPipe Hands and drawing utilities
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

sequence = []
sequence_length = 1  # Set sequence length to 10 frames for rolling window approach - not working yet

while cap.isOpened():
    ret, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        landmarks = []
        for lm in results.multi_hand_landmarks[0].landmark:
            landmarks.extend([lm.x, lm.y, lm.z])

        # Draw hand landmarks on the frame
        mp_drawing.draw_landmarks(
            frame, 
            results.multi_hand_landmarks[0], 
            mp_hands.HAND_CONNECTIONS
        )

        # Append new frame landmarks to sequence
        sequence.append(landmarks)
        if len(sequence) > sequence_length:
            sequence.pop(0)

        # Predict only if the sequence is full
        if len(sequence) == sequence_length:
            sequence_input = np.array(sequence).flatten()[np.newaxis, ..., np.newaxis]
            prediction = model.predict(sequence_input)
            predicted_label_index = np.argmax(prediction)
            predicted_label = label_encoder.inverse_transform([predicted_label_index])
            confidence = prediction[0][predicted_label_index] * 100  # Get confidence percentage

            # Display prediction and confidence
            cv2.putText(frame, f"Predicted: {predicted_label[0]} ({confidence:.2f}%)", 
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)  # Black text color
    
    # Display the frame
    cv2.imshow("ASL Sign Language Detection", frame)
    
    # Press 'C' to terminate the loop
    if cv2.waitKey(1) & 0xFF == ord("c"):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 21ms/step
